In [2]:
from carla import Benchmark
import carla.evaluation.catalog as evaluation_catalog
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog
import shap 
from sklearn.preprocessing import OrdinalEncoder
from carla.data.causal_model import CausalModel
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from carla.data.catalog import CsvCatalog
import numpy as np 

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2


/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [3]:
import pandas as pd
# Load Data 
dataframe = pd.read_csv('./census.csv', index_col=['Unnamed: 0'])
continuous = dataframe.drop(columns=['target']).columns

dataset = CsvCatalog(file_path="census.csv",
                     continuous=continuous,
                     categorical=[],
                     immutables=[],
                     target='target')

In [4]:
#Model 

training_params = {"lr": 0.01, "epochs": 10, "batch_size": 16, "hidden_size": [18, 9, 2]}

ml_model = MLModelCatalog(
    dataset, model_type="ann", load_online=False, backend="pytorch"
)
ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"],
    force_train=True
)


balance on test set 0.24156429156429157, balance on test set 0.2385456332145928
Epoch 0/9
----------
train Loss: 0.4017 Acc: 0.8102

test Loss: 0.4050 Acc: 0.8124

Epoch 1/9
----------
train Loss: 0.3556 Acc: 0.8320

test Loss: 0.5835 Acc: 0.6692

Epoch 2/9
----------
train Loss: 0.3484 Acc: 0.8358

test Loss: 0.3911 Acc: 0.8262

Epoch 3/9
----------
train Loss: 0.3455 Acc: 0.8382

test Loss: 0.3383 Acc: 0.8419

Epoch 4/9
----------
train Loss: 0.3420 Acc: 0.8404

test Loss: 0.3798 Acc: 0.8241

Epoch 5/9
----------
train Loss: 0.3415 Acc: 0.8397

test Loss: 0.3337 Acc: 0.8461

Epoch 6/9
----------
train Loss: 0.3393 Acc: 0.8410

test Loss: 0.3402 Acc: 0.8360

Epoch 7/9
----------
train Loss: 0.3393 Acc: 0.8407

test Loss: 0.3297 Acc: 0.8473

Epoch 8/9
----------
train Loss: 0.3376 Acc: 0.8401

test Loss: 0.3445 Acc: 0.8477

Epoch 9/9
----------
train Loss: 0.3362 Acc: 0.8408

test Loss: 0.3331 Acc: 0.8449



In [5]:
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:20]

display(test_factual)

,Unnamed: 0,Age,Workclass,Education-Num,Marital Status,...,Capital Gain,Capital Loss,Hours per week,Country,target
0,0,0.301370,0.875,0.800000,0.666667,...,0.02174,0.0,0.397959,0.951220,False
1,1,0.452055,0.750,0.800000,0.333333,...,0.00000,0.0,0.122449,0.951220,False
2,2,0.287671,0.500,0.533333,0.000000,...,0.00000,0.0,0.397959,0.951220,False
3,3,0.493151,0.500,0.400000,0.333333,...,0.00000,0.0,0.397959,0.951220,False
6,6,0.438356,0.500,0.266667,0.500000,...,0.00000,0.0,0.153061,0.560976,False
7,7,0.479452,0.750,0.533333,0.333333,...,0.00000,0.0,0.448980,0.951220,True
11,11,0.178082,0.875,0.800000,0.333333,...,0.00000,0.0,0.397959,0.463415,True
12,12,0.082192,0.500,0.800000,0.666667,...,0.00000,0.0,0.295918,0.951220,False
13,13,0.205479,0.500,0.733333,0.666667,...,0.00000,0.0,0.500000,0.951220,False
14,14,0.315068,0.500,0.666667,0.333333,...,0.00000,0.0,0.397959,0.000000,True


In [6]:
hyperparams = {"loss_type": "BCE", "binary_cat_features": True}

recourse_method = recourse_catalog.Wachter(ml_model, hyperparams)
df_cfs = recourse_method.get_counterfactuals(test_factual)

display(df_cfs)

[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactua

,Age,Capital Gain,Capital Loss,Country,Education-Num,...,Occupation,Race,Relationship,Sex,Workclass
0,0.311370,0.031740,0.010000,0.961219,0.810000,...,0.081429,1.010000,0.190000,0.990000,0.865000
1,0.432132,0.019929,0.019924,0.931479,0.819926,...,0.305586,1.019620,-0.019165,0.980100,0.733346
2,0.327290,0.039664,0.039648,0.911597,0.572994,...,0.418259,0.960370,0.160387,1.039645,0.460424
3,0.497894,0.039017,0.038691,0.913048,0.438506,...,0.467968,0.528517,-0.027334,0.966417,0.473702
6,0.497616,0.097125,0.096461,0.463758,0.362831,...,0.669305,0.591714,0.299493,0.099424,0.508568
7,0.459537,0.019921,0.019916,0.931508,0.553252,...,0.305572,1.019577,-0.019065,0.980110,0.733724
11,0.188082,0.010000,0.010000,0.473415,0.810000,...,0.724286,0.260000,-0.010000,0.990000,0.865000
12,0.122006,0.039815,0.039811,0.982999,0.839814,...,0.111208,1.039798,0.639801,0.039802,0.539712
13,0.235261,0.029457,0.029653,0.979778,0.762830,...,0.885413,0.529824,0.170163,0.970543,0.471448
14,0.325068,0.010000,0.010000,-0.010000,0.676667,...,0.224286,0.260000,-0.010000,0.990000,0.490000


In [7]:
from carla.evaluation import remove_nans
from carla.evaluation.api import Evaluation
class Sematic(Evaluation):
    """
    """

    def __init__(self, mlmodel, causal_graph):
        super().__init__()
        self.mlmodel= ml_model
        self.causal_graph=causal_graph
    def evaluate(self,factual: np.ndarray, counterfactual: np.ndarray, causal_model):
        # generate data 
        cf_label=self.ml_model.predict(counterfactual)
        causal_label = None
        if cf_label ==causal_label:
            return 1 
        else: 
            return 0

''' 
Analysis
Probitlity Distributions 
Histogram True Positive, False Positives 
'''
'''
Parts of counterfactuals actually change 
1hop and 2 hop relations 
'''
'''Histogram Features / true Positive / false Positive'''
# first initialize the benchmarking class by passing
# black-box-model, recourse method, and factuals into it
#benchmark = Benchmark(ml_model, recourse_method, factuals)

# now you can decide if you want to run all measurements
# or just specific ones.
#evaluation_measures = [
#    evaluation_catalog.YNN(benchmark.mlmodel, {"y": 5, "cf_label": 1}),
#    evaluation_catalog.Distance(benchmark.mlmodel),
#    evaluation_catalog.SuccessRate(),
#    evaluation_catalog.Redundancy(benchmark.mlmodel, {"cf_label": 1}),
#    evaluation_catalog.ConstraintViolation(benchmark.mlmodel),
#    evaluation_catalog.AvgTime({"time": benchmark.timer}),
#]

# now run all implemented measurements and create a
# DataFrame which consists of all results
#results = benchmark.run_benchmark(evaluation_measures)


'Histogram Features / true Positive / false Positive'